# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

## 1. Ten Countries with the Lowest Infant Mortality Rates

In [211]:
document = ET.parse( './data/mondial_database.xml' )

def read_countries(document):
    countries = []
    for element in document.iterfind('country'):
        name = element.find('name').text
        infant_mortality = element.find('infant_mortality')
        cities = []
        for city_element in element.getiterator('city'):
            city = {'name' : city_element.find('name').text}
            for population_element in city_element.getiterator('population'):
                population_year = population_element.attrib['year']
                population_num = int(population_element.text)
                city['year']= population_year
                city['population'] = population_num
            cities.append(city)
        countries.append({'name' : name,
                          'cities' : cities,
                          'infant_mortality' : infant_mortality.text if infant_mortality is not None else None})
    return countries

document = ET.parse( './data/mondial_database.xml' )
countries = read_countries(document)

import pandas as pd
countries_df = pd.DataFrame(countries)
countries_df.dropna(inplace=True)
countries_df['infant_mortality'] = pd.to_numeric(countries_df['infant_mortality'])
countries_df.nsmallest(10, 'infant_mortality')[['name', 'infant_mortality']]


,name,infant_mortality
38,Monaco,1.81
98,Japan,2.13
36,Norway,2.48
117,Bermuda,2.48
106,Singapore,2.53
37,Sweden,2.60
10,Czech Republic,2.63
78,Hong Kong,2.73
79,Macao,3.13
44,Iceland,3.15


## 2. Ten Cities with the Largest Population

In [214]:
df = pd.DataFrame()
for country in countries:
    city_df = json_normalize(country, 'cities') # Normalize JSON
    city_df['country'] = country['name']
    df = df.append(city_df)

# Drop rows with missing values
df = df.dropna(axis=0)

# Use the maximum value of the census count for each city as the
# basis for comparison, since some of the values for population are missing.
# This assumes a positive trend in population, which should be true for most large cities.
# We could also use interpolation to replace the missing values (see section below).
max_populations_by_city = df.groupby('name')['population'].max()

cities = max_populations_by_city.index.values
population = max_populations_by_city.values
population
cities_df = pd.DataFrame({'city' : cities, 
                          'population' : population})
cities_df.sort_values(by='population', 
                      ascending=False)[:10]


,city,population
2380,Shanghai,22315474.0
1127,Istanbul,13710512.0
1742,Mumbai,12442373.0
1729,Moskva,11979529.0
283,Beijing,11716620.0
2535,São Paulo,11152344.0
2631,Tianjin,11090314.0
941,Guangzhou,11071424.0
686,Delhi,11034555.0
2393,Shenzhen,10358381.0


## Top 10 Cities by Population: With Interpolation of missing values

The results are the same whether we use interpolation or use the maximum population for each city:

In [279]:
df = pd.DataFrame()
for country in countries:
    city_df = json_normalize(country, 'cities') # Normalize JSON
    city_df['country'] = country['name']
    df = df.append(city_df)

# Remove rows with missing years
df = df.dropna(subset=['year'])
df = df.set_index(['name', 'year'])

df=df.interpolate(method='linear')
df.sort_values(by='population', 
               ascending=False)[:10]

,,country,population
name,year,,
Shanghai,2010,China,22315474.0
Istanbul,2012,Turkey,13710512.0
Mumbai,2011,India,12442373.0
Moskva,2013,Russia,11979529.0
Beijing,2010,China,11716620.0
São Paulo,2010,Brazil,11152344.0
Tianjin,2010,China,11090314.0
Guangzhou,2010,China,11071424.0
Delhi,2011,India,11034555.0
